## Feature Engineering

We'll generate average frequency content over the length of the sample, which should be useful in separating instruments by the strength of their harmonics.

### Create Tables for Harmonic Content of Samples

In [3]:
import config
import mysql
import mysql.connector
from mysql.connector import errorcode

# DB FUNCTIONS

def connect(db_name=None):
    if db_name:
        cnx = mysql.connector.connect(
            host = config.rds_host,
            user = config.rds_user,
            passwd = config.rds_password,
            database = db_name)
    else:
        cnx = mysql.connector.connect(
            host = config.rds_host,
            user = config.rds_user,
            passwd = config.rds_password)
    cursor = cnx.cursor()
    return cursor, cnx


In [ ]:
create_frequency_table_query = '''CREATE TABLE frequencies (
    frequency_id INT NOT NULL,
    frequency DOUBLE NOT NULL,
    log_2_frequency DOUBLE NOT NULL,
    PRIMARY KEY (frequency_id))'''

cur, cnx = connect('sound_shift')
cur.execute(create_frequency_table_query)
cnx.close()

In [ ]:
import librosa
from math import log

sr = 44100
n_fft = 4096
freqs = librosa.core.fft_frequencies(sr=sr, n_fft=n_fft)[1:]

values = []

for idx, freq in enumerate(freqs):
    values.append( (idx, float(freq), log(freq, 2)) )

frequency_insert_query = '''INSERT INTO frequencies (frequency_id, frequency, log_2_frequency)
                            VALUES (%s, %s, %s)'''
    
cur, cnx = connect('instruments')
cur.executemany(frequency_insert_query, values)
cnx.commit()
cnx.close()

In [ ]:
create_harmonic_content_table_query = '''CREATE TABLE harmonic_content (
                                        sample_id INT NOT NULL,
                                        frequency_id INT NOT NULL,
                                        amplitude DOUBLE NOT NULL,
                                        FOREIGN KEY (sample_id) REFERENCES samples(sample_id),
                                        FOREIGN KEY (frequency_id) REFERENCES frequencies(frequency_id))'''

cur, cnx = connect('instruments')
cur.execute(create_harmonic_content_table_query)
cnx.close()

### Create Table for Chroma Content

In [12]:
import config
import mysql
import mysql.connector
from mysql.connector import errorcode

# DB FUNCTIONS

def connect(db_name=None):
    if db_name:
        cnx = mysql.connector.connect(
            host = config.rds_host,
            user = config.rds_user,
            passwd = config.rds_password,
            database = db_name)
    else:
        cnx = mysql.connector.connect(
            host = config.rds_host,
            user = config.rds_user,
            passwd = config.rds_password)
    cursor = cnx.cursor()
    return cursor, cnx


In [14]:
create_chroma_table_query = '''CREATE TABLE chroma (
                                        sample_id INT NOT NULL,
                                        predicted_tonic INT NOT NULL,
                                        amp0 DOUBLE NOT NULL,
                                        amp1 DOUBLE NOT NULL,
                                        amp2 DOUBLE NOT NULL,
                                        amp3 DOUBLE NOT NULL,
                                        amp4 DOUBLE NOT NULL,
                                        amp5 DOUBLE NOT NULL,                                        
                                        amp6 DOUBLE NOT NULL,
                                        amp7 DOUBLE NOT NULL,
                                        amp8 DOUBLE NOT NULL,                                        
                                        amp9 DOUBLE NOT NULL,
                                        amp10 DOUBLE NOT NULL,
                                        amp11 DOUBLE NOT NULL,                                        
                                        FOREIGN KEY (sample_id) REFERENCES samples(sample_id))'''

cur, cnx = connect('instruments')
cur.execute(create_chroma_table_query)
cnx.close()

### Create Table for Feature Soup

In [34]:
import config
import mysql
import mysql.connector
from mysql.connector import errorcode

# DB FUNCTIONS

def connect(db_name=None):
    if db_name:
        cnx = mysql.connector.connect(
            host = config.rds_host,
            user = config.rds_user,
            passwd = config.rds_password,
            database = db_name)
    else:
        cnx = mysql.connector.connect(
            host = config.rds_host,
            user = config.rds_user,
            passwd = config.rds_password)
    cursor = cnx.cursor()
    return cursor, cnx


In [36]:
create_feature_soup_table_query = f"""CREATE TABLE feature_soup (sample_id INT NOT NULL,
                                    {' DOUBLE NOT NULL, '.join([f'feat{str(x)}' for x in range(112)])} DOUBLE NOT NULL,
                                    FOREIGN KEY (sample_id) REFERENCES samples(sample_id))"""
    

cur, cnx = connect('instruments')
cur.execute(create_feature_soup_table_query)
cnx.close()

### Extract Harmonic Content from Samples to Populate Table

In [4]:
import config
import logging
import boto3
from botocore.exceptions import ClientError

def download_file(object_name, bucket='instrument-samples-1337', file_name=None):
    """Download a file from an S3 bucket

    :param object_name: S3 object to download
    :param bucket: Bucket to download from
    :param file_name: file name to download to. If not specified then object_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if file_name is None:
        file_name = object_name

    # Upload the file
    s3_client = boto3.client('s3',
                             aws_access_key_id=config.aws_access_key_id,
                             aws_secret_access_key=config.aws_secret_access_key)
    try:
        response = s3_client.download_file(bucket, object_name, file_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [66]:
import pandas as pd
import librosa.core as lrc
import librosa
import os
import numpy as np
import matplotlib.pyplot as plt
from math import log
from IPython.display import display

def perform_stft(sample_id, sample, sample_rate):
        n_fft = 4096
        stft = librosa.stft(sample, n_fft=n_fft)[1:] # Short Time Fourier Transform
        fig, ax = plt.subplots(figsize=(20,5))
        freqs = librosa.core.fft_frequencies(sr=sample_rate, n_fft=n_fft)[1:]
        log_2_freqs = list(map(lambda x: log(x, 2), freqs))
        ax.plot(log_2_freqs, np.abs(stft).mean(axis=1))
        ax.set_xlabel('log_2 of frequency')
        ax.set_ylabel('average amplitude')
        display(fig)
        plt.close(fig)
        values = []
        print('Inserting harmonic content into database...')
        for idx, value in enumerate(np.abs(stft).mean(axis=1)):
            values.append( (sample_id, idx, float(value)) )
        query = f"""INSERT INTO harmonic_content (sample_id, frequency_id, amplitude)
                    VALUES (%s, %s, %s)"""
        return query, values
    
def perform_chroma(sample_id, sample, sample_rate):
    print('Extracting chroma features...')
    chroma = librosa.feature.chroma_stft(y=sample, sr=sample_rate)
    vals = list(chroma.mean(axis=1))
    note_max = np.argmax(vals)
    vals_shifted = {}
    for i in range(12):
        vals_shifted[i] = vals[(i+note_max) % 12]
    vals2 = list(vals_shifted.values())
    fig, ax = plt.subplots()
    ax.bar([0,1,2,3,4,5,6,7,8,9,10,11], vals2)
    display(fig)
    plt.close(fig)
    print('Inserting chroma features into database...')
    values_pre = [int(sample_id), int(note_max)] + [float(x) for x in vals2]
    values = tuple(values_pre)
    query = f"""INSERT INTO chroma (sample_id, predicted_tonic, amp0, amp1, amp2,
                amp3, amp4, amp5, amp6, amp7, amp8, amp9, amp10, amp11)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    return query, [values]

def perform_feature_soup(sample_id, sample, sample_rate):
    y=sample
    sr=sample_rate
    onset = librosa.onset.onset_strength(y=y, sr=sr)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    flatness = librosa.feature.spectral_flatness(y=y)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    tonnetz = librosa.feature.tonnetz(y=y, sr=sr)
                                  
    frame = np.argmax(onset) if np.argmax(onset) + 5 < onset.shape[0] else 5
    engineered_dims = []
    for feat in [chroma.T, centroid.T, contrast.T, flatness.T, rolloff.T, tonnetz.T]:
        for j in [0, 1, 2, 5]:
            for k in range(feat.shape[1]):
                engineered_dims.append(float(feat[frame+j][k]))
    query = f"""INSERT INTO feature_soup (sample_id, {', '.join([f'feat{str(x)}' for x in range(len(engineered_dims))])})
                VALUES (%s, {', '.join(['%s' for x in range(len(engineered_dims))])})"""
    return query, [tuple([sample_id] + engineered_dims)]
    
def process_sample(sample_id, processing_function):
    cur, cnx = connect('instruments')
    cur.execute(f"""SELECT * FROM instruments.samples
                    WHERE sample_id = {sample_id}""")
    df = pd.DataFrame(cur.fetchall())
    df.columns = [i[0] for i in cur.description]
    sample_metadata = df.iloc[0]
    print('Attemping to download the following sample:')
    print(sample_metadata)
    sample_id = int(sample_metadata['sample_id'])
    sample_file_ext = sample_metadata['file_extension']
    object_name = f'{str(sample_id)}.{sample_file_ext}'
    print(object_name)
    if download_file(object_name):
        print('Analyzing sample...')
        y, sr = lrc.load(object_name, sr=None)
        query, values = processing_function(sample_id, y, sr)
        cur.executemany(query, values)
        os.remove(object_name)
        cnx.commit()
        cnx.close()
        print('Done!')
    else:
        print('Download failed.')
        cnx.close()

In [67]:
for i in range(213,2338):
    process_sample(i, perform_feature_soup)

Attemping to download the following sample:
sample_id                 213
instrument_name    EbClarinet
note                       D4
expression                   
source               Iowa2012
file_extension            aif
Name: 0, dtype: object
213.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id                 214
instrument_name    EbClarinet
note                       G3
expression                   
source               Iowa2012
file_extension            aif
Name: 0, dtype: object
214.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id                 215
instrument_name    EbClarinet
note                      Bb3
expression                   
source               Iowa2012
file_extension            aif
Name: 0, dtype: object
215.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id                 216
instrument_name    EbClarinet
note                      Eb4
expression             

Analyzing sample...
Done!
Attemping to download the following sample:
sample_id                 243
instrument_name    BbClarinet
note                      Db5
expression                   
source               Iowa2012
file_extension            aif
Name: 0, dtype: object
243.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id                 244
instrument_name    BbClarinet
note                       C4
expression                   
source               Iowa2012
file_extension            aif
Name: 0, dtype: object
244.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id                 245
instrument_name    BbClarinet
note                       E6
expression                   
source               Iowa2012
file_extension            aif
Name: 0, dtype: object
245.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id                 246
instrument_name    BbClarinet
note                      A

Analyzing sample...
Done!
Attemping to download the following sample:
sample_id                   273
instrument_name    BassClarinet
note                         D4
expression                     
source                 Iowa2012
file_extension              aif
Name: 0, dtype: object
273.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id                   274
instrument_name    BassClarinet
note                        Bb4
expression                     
source                 Iowa2012
file_extension              aif
Name: 0, dtype: object
274.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id                   275
instrument_name    BassClarinet
note                        Eb4
expression                     
source                 Iowa2012
file_extension              aif
Name: 0, dtype: object
275.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id                   276
instrument_name    

Analyzing sample...
Done!
Attemping to download the following sample:
sample_id                   301
instrument_name    BassClarinet
note                         D5
expression                     
source                 Iowa2012
file_extension              aif
Name: 0, dtype: object
301.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id                   302
instrument_name    BassClarinet
note                        Bb5
expression                     
source                 Iowa2012
file_extension              aif
Name: 0, dtype: object
302.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id                   303
instrument_name    BassClarinet
note                        Eb5
expression                     
source                 Iowa2012
file_extension              aif
Name: 0, dtype: object
303.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id                   304
instrument_name    

Analyzing sample...
Done!
Attemping to download the following sample:
sample_id               330
instrument_name     Bassoon
note                     C4
expression                 
source             Iowa2012
file_extension          aif
Name: 0, dtype: object
330.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id               331
instrument_name     Bassoon
note                    Ab3
expression                 
source             Iowa2012
file_extension          aif
Name: 0, dtype: object
331.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id               332
instrument_name     Bassoon
note                    Db5
expression                 
source             Iowa2012
file_extension          aif
Name: 0, dtype: object
332.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id               333
instrument_name     Bassoon
note                     F4
expression                 
source     

...

Analyzing sample...
Done!
Attemping to download the following sample:
sample_id                2223
instrument_name    Vibraphone
note                       D3
expression                bow
source               Iowa2012
file_extension            aif
Name: 0, dtype: object
2223.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id                2224
instrument_name    Vibraphone
note                       F5
expression                bow
source               Iowa2012
file_extension            aif
Name: 0, dtype: object
2224.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id                2225
instrument_name    Vibraphone
note                      Eb4
expression                bow
source               Iowa2012
file_extension            aif
Name: 0, dtype: object
2225.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id                2226
instrument_name    Vibraphone
note                    

/Users/flatrionschool/anaconda3/lib/python3.7/site-packages/librosa/core/pitch.py:146: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


Done!
Attemping to download the following sample:
sample_id              2249
instrument_name       bells
note                    Ab6
expression          plastic
source             Iowa2012
file_extension          aif
Name: 0, dtype: object
2249.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id              2250
instrument_name       bells
note                     B5
expression          plastic
source             Iowa2012
file_extension          aif
Name: 0, dtype: object
2250.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id              2251
instrument_name       bells
note                     D7
expression          plastic
source             Iowa2012
file_extension          aif
Name: 0, dtype: object
2251.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id              2252
instrument_name       bells
note                    Db5
expression          plastic
source             Iowa2012


Analyzing sample...
Done!
Attemping to download the following sample:
sample_id              2280
instrument_name       bells
note                    Eb8
expression            brass
source             Iowa2012
file_extension          aif
Name: 0, dtype: object
2280.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id              2281
instrument_name       bells
note                     A5
expression            brass
source             Iowa2012
file_extension          aif
Name: 0, dtype: object
2281.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id              2282
instrument_name       bells
note                     G7
expression            brass
source             Iowa2012
file_extension          aif
Name: 0, dtype: object
2282.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id              2283
instrument_name       bells
note                    Eb5
expression            brass
source  

Analyzing sample...
Done!
Attemping to download the following sample:
sample_id              2311
instrument_name       bells
note                    Gb7
expression            brass
source             Iowa2012
file_extension          aif
Name: 0, dtype: object
2311.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id              2312
instrument_name       bells
note                     F7
expression            brass
source             Iowa2012
file_extension          aif
Name: 0, dtype: object
2312.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id              2313
instrument_name     Crotale
note                     C7
expression                 
source             Iowa2012
file_extension          aif
Name: 0, dtype: object
2313.aif
Analyzing sample...
Done!
Attemping to download the following sample:
sample_id              2314
instrument_name     Crotale
note                    Db6
expression                 
source  